In [24]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [ ]:
alphabet = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"," ","."]

with open('data/sample/ciphertext.txt', 'r', encoding='utf-8') as f:
    ciphertext = f.read()

P = np.loadtxt("data/letter_probabilities.csv", delimiter = ",")
M = np.loadtxt("data/letter_transition_matrix.csv", delimiter = ",")

inverse_perm = {c: i for i, c in enumerate(permutation)}
print(inverse_perm)

print(M)

# r to m

{'i': 0, 'u': 1, 'x': 2, 'q': 3, 'g': 4, 't': 5, 'm': 6, 'h': 7, ' ': 8, 'k': 9, 'w': 10, 'a': 11, '.': 12, 'b': 13, 'v': 14, 'l': 15, 'p': 16, 'e': 17, 'r': 18, 'j': 19, 's': 20, 'z': 21, 'o': 22, 'c': 23, 'f': 24, 'y': 25, 'd': 26, 'n': 27}
[[5.04501242e-04 5.54097490e-02 1.13551483e-01 3.02411821e-02
  4.99380177e-02 7.80910625e-02 5.94468061e-02 1.89907131e-01
  1.45998388e-02 1.57575350e-01 4.74800705e-03 7.02785129e-02
  1.32769720e-01 1.55120223e-02 4.70280728e-03 1.19610115e-01
  9.99720078e-06 5.46289060e-02 4.82646730e-02 3.21873099e-02
  2.25309904e-02 5.22355079e-02 2.27838623e-01 7.23050493e-02
  4.54345365e-03 3.82135348e-01 1.15055016e-01 0.00000000e+00]
 [2.17184988e-02 5.09415164e-03 6.45619682e-05 7.83372126e-04
  9.36574162e-04 1.54428315e-04 1.19349347e-03 1.82364905e-03
  6.69256590e-03 9.99720078e-06 9.92882223e-04 7.78414434e-04
  1.44261630e-02 4.54380026e-04 7.80803481e-03 1.55132346e-03
  9.99720078e-06 1.90834650e-03 1.42624638e-03 3.10565829e-04
  1.59431740

In [ ]:
#MH MCMC Algorithm
def log_likelihood(perm, txt):
    inverse_perm = {c: i for i, c in enumerate(perm)}

    log_lik = 0

    for i, char in enumerate(txt):
        decoded_char_idx = inverse_perm[char]

        if i == 0:
            log_lik += np.log(P[decoded_char_idx])

        else:
            prev_decoded_char_idx = inverse_perm[txt[i-1]]

            if M[decoded_char_idx][prev_decoded_char_idx] == 0:
                return float('-inf')

            log_lik += np.log(M[decoded_char_idx][prev_decoded_char_idx])

    return log_lik

def propose_new_permutation(perm):
    i, j = random.sample(range(len(perm)), 2)
    new_perm = perm.copy()
    new_perm[i], new_perm[j] = new_perm[j], new_perm[i]
    return new_perm


def MH(T):
    permutation = random.sample(alphabet, len(alphabet))
    f_star = permutation
    L_star = log_likelihood(permutation, ciphertext)

    f_prev = permutation

    for i in range(T):
        fprime = propose_new_permutation(f_prev)

        old_ll = log_likelihood(f_prev, ciphertext)
        new_ll = log_likelihood(fprime, ciphertext)

        if old_ll == float('-inf'):
            bernoulli_param = 1
        else:
            bernoulli_param = min(1, np.exp(new_ll-old_ll))

        if random.random() < bernoulli_param:

            if new_ll > L_star:
                L_star = new_ll
                f_star = fprime

            f_prev = fprime

    return L_star, f_star
    

(-inf, ['u', '.', 'i', 'q', 'y', 'a', 'd', 'c', 'g', 'l', 'f', 'm', 'v', 'r', 'e', 'x', 'z', 'n', 'k', ' ', 'h', 't', 's', 'o', 'p', 'j', 'b', 'w'])
